In [1]:
import pandas as pd

In [2]:
import altair as alt

In [6]:
from altair import expr

In [7]:
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)

In [8]:
data = pd.read_excel("daily_increment_positif.xlsx")

In [9]:
data = data.set_index("tanggal")

In [10]:
data["total"] = data.sum(1)

In [11]:
prov_list = ['aceh','bali','banten','bangka belitung','bengkulu','yogyakarta','jakarta','jambi','jawa barat','jawa tengah','jawa timur','kalimantan barat','kalimantan timur','kalimantan tengah','kalimantan selatan','kalimantan utara','kepulauan riau','nusa tenggara barat','sumatera selatan','sumatera barat','sulawesi utara','sumatera utara','sulawesi tenggara','sulawesi selatan','sulawesi tengah','lampung','riau','maluku utara','maluku','papua barat','papua','sulawesi barat','nusa tenggara timur','gorontalo'
            ]

In [12]:
prov = data[prov_list]

In [13]:
prov_long = prov.reset_index().melt(id_vars="tanggal")

In [14]:
selection = alt.selection_multi(fields=['variable'])

In [15]:
selection2 = alt.selection_multi(fields=['variable'])

In [16]:
base = alt.Chart(prov_long)

In [17]:
cumsumbase = base.mark_line().transform_window(
    sort=[{'field': 'tanggal'}],
    frame=[None, 0],
    cumulative_sum='sum(value)',
    groupby = ['variable']
)

In [18]:
cumsumlinear = cumsumbase.encode(
    x='tanggal:T',
    y = alt.Y('cumulative_sum:Q',scale=alt.Scale(type="linear"),title = 'kasus'),
    color = alt.Color('variable:N',legend=None)
).transform_filter(selection & selection2)

In [19]:
cumsumlog = cumsumbase.encode(
    x='tanggal:T',
    y = alt.Y('cum_log:Q',scale=alt.Scale(type="log"),title = 'kasus'),
    color = alt.Color('variable:N',legend=None)
).transform_filter(selection & selection2).transform_calculate(cum_log = expr.if_(datum.cumulative_sum == 0,expr.NaN,datum.cumulative_sum))

In [20]:
bar_base = base.mark_bar().transform_aggregate(prov_sum = 'sum(value)',groupby=['variable']).encode(
    y = alt.Y('variable:N',sort='-x',title = 'provinsi'),
    x = alt.X('prov_sum:Q',title = 'kasus'),
    color = 'variable:N')
bar = bar_base.add_selection(selection)

In [21]:
mark = bar_base.mark_text(dx=15 ).encode(text = 'prov_sum:Q',color = alt.value('black')).add_selection(selection2)

In [22]:
charts = (cumsumlinear & cumsumlog) |(bar + mark) 

In [23]:
charts.properties(
    title='Indonesia Covid-19'
)

alt.HConcatChart(...)

In [24]:
charts.save('indonesia_covid.json')